<a href="https://colab.research.google.com/github/vijayakanth/Transfer-Learning/blob/main/99_58_Xception_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
import keras
import glob
import cv2
import os

#TRAINGING SET:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def getImages(no_of_classes=6):
    disease_images = []
    labels = []
    i = 0
    for pest_image_path in glob.glob("/content/drive/MyDrive/ColabNotebooks/data/teaLeaveDisease/*"):
        insect_label = pest_image_path.split("/")[-1]
        if no_of_classes == i:
            break
        for image_path in glob.glob(os.path.join(pest_image_path, "*.jpg")):
            # image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (299, 299),interpolation = cv2.INTER_CUBIC)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            disease_images.append(image)
            labels.append(insect_label)
        i = i + 1

    disease_images = np.array(disease_images)
    labels = np.array(labels)
    label_to_id_dict = {v: i for i, v in enumerate(np.unique(labels))}
    id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

    label_ids = np.array([label_to_id_dict[x] for x in labels])
    disease_images.shape, label_ids.shape, labels.shape

    print ("Total Images : " +  str(disease_images.shape[0]))
    # print ("Total Labels : " +  str(label_ids))
    return disease_images, label_ids

In [ ]:
disease, labels = getImages(no_of_classes=6)

Total Images : 1200


In [ ]:
#data augmentation
def Augmentation(X_train,X_valid,X_test,Y_train,Y_valid,Y_test,batch_size,test_batchSize):

        #Augmentation ,we will use for training and not in validation and testing
        train_datagen= ImageDataGenerator(rescale=1./255,
                                          rotation_range=30,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')

        valid_datagen = ImageDataGenerator(rescale=1. / 255)
        test_datagen = ImageDataGenerator(rescale=1. / 255)

        train_batch = train_datagen.flow(X_train,
                                         Y_train,
                                         batch_size=batch_size,shuffle=True)

        valid_batch = valid_datagen.flow(X_valid,
                                         Y_valid,
                                         batch_size=batch_size,shuffle=True)

        test_batch = test_datagen.flow(X_test,
                                       Y_test,
                                       batch_size=test_batchSize,shuffle=False)

        return (train_batch,valid_batch,test_batch)

In [ ]:
#SETTING UP InsceptionV3
def model(X_train, X_valid,X_test, Y_train, Y_valid, Y_test,Y,numClasses):
    from tensorflow.keras.applications import Xception
    from keras.layers import Dense, GlobalAveragePooling2D,Dropout, Flatten, Input
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from keras.models import Sequential
    from keras.models import Model

    # Load pre-trained Xception model
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

    # # Freeze initial layers of the base model
    # for layer in base_model.layers:
    #     layer.trainable = False

    # # Identify the number of layers to unfreeze
    num_layers_to_unfreeze = 1  # Adjust this based on your preference

    # Unfreeze the last 'num_layers_to_unfreeze' layers
    for layer in base_model.layers[-num_layers_to_unfreeze:]:
        layer.trainable = True

    # Add custom top layers for classification
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x=Dropout(rate=0.3)(x)
    x = Dense(128, activation='relu')(x)
    x=Dropout(rate=0.3)(x)
    output = Dense(6, activation='softmax')(x)  # 6 classes

    # Create the model
    model = Model(inputs=base_model.input, outputs=output)

    # from tensorflow.keras.optimizers import SGD
    # model.compile(optimizer=SGD(lr=0.0001, momentum=0.90), loss='categorical_crossentropy',metrics=['accuracy'])

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['accuracy'])

    batch_size = 32
    test_batchSize=1
    epochs = 100
    train_batch, valid_batch,test_batch = Augmentation(X_train,X_valid, X_test, Y_train, Y_valid, Y_test,batch_size,test_batchSize)

    model.fit(train_batch,
              steps_per_epoch= X_train.shape[0]//batch_size,
              validation_data=valid_batch,
              epochs=epochs,
              validation_steps=X_valid.shape[0]//batch_size,
              verbose=2)

    test_accuracy=model.evaluate(test_batch,steps=len(test_batch))
    print('Testing Accuracy Score :',test_accuracy)

    return test_accuracy

In [ ]:
from sklearn.model_selection  import train_test_split,KFold
numClasses=6
train,test,trLabels,teLabels=train_test_split(disease,labels,test_size=0.2, random_state=42)
X_test = test
Y_test = teLabels
Y=Y_test
Y_test = keras.utils.to_categorical(Y_test,numClasses)
def KFoldCV(Images, Labels, numClasses):
    # KFold Cross Validation approach
    kf = KFold(n_splits=5,shuffle=True,random_state=42)
    kf.split(Images)

    # Initialize the test accuracy array. The testing accuracy of each model will be added to this array
    test_accuracy_model = []
    Fold=1
    # Iterate over each train-test split
    for train_index, valid_index in kf.split(Images):
          print('Fold No:', Fold)
        # Split training:validation
          X_train, X_valid = Images[train_index], Images[valid_index]
          Y_train, Y_valid = Labels[train_index], Labels[valid_index]

        #One Hot Encode the Output
          Y_train = keras.utils.to_categorical(Y_train,numClasses)
          Y_valid = keras.utils.to_categorical(Y_valid,numClasses)

        # Train the model
          print('Original Sizes:', X_train.shape,X_valid.shape, X_test.shape,Y_train.shape,Y_valid.shape,Y_test.shape)
          score=model(X_train, X_valid,X_test, Y_train, Y_valid, Y_test,Y,numClasses)
          test_accuracy_model.append(score[1]*100)

          Fold=Fold+1
    return (test_accuracy_model)

#  8-Fold 6 Classes


In [ ]:
import time
start_time= time.time()
accurary = KFoldCV(train,trLabels, 6)
end_time=time.time()
print(accurary)
print("%s %.2f%%" % ("Average Accuracy: ",np.mean(accurary)))
print("%s %.2f%%" % ("Standard Deviation: +/-", np.std(accurary)))
print("Total time taken {}".format(end_time-start_time))


Fold No: 1
Original Sizes: (768, 299, 299, 3) (192, 299, 299, 3) (240, 299, 299, 3) (768, 6) (192, 6) (240, 6)
83683744/83683744 [==============================] - 3s 0us/step
Epoch 1/100
24/24 - 46s - loss: 1.5345 - accuracy: 0.4427 - val_loss: 1.2840 - val_accuracy: 0.5469 - 46s/epoch - 2s/step
Epoch 2/100
24/24 - 16s - loss: 0.7534 - accuracy: 0.8203 - val_loss: 0.5362 - val_accuracy: 0.7969 - 16s/epoch - 657ms/step
Epoch 3/100
24/24 - 16s - loss: 0.2865 - accuracy: 0.9271 - val_loss: 0.3361 - val_accuracy: 0.8906 - 16s/epoch - 677ms/step
Epoch 4/100
24/24 - 16s - loss: 0.1677 - accuracy: 0.9583 - val_loss: 0.1659 - val_accuracy: 0.9427 - 16s/epoch - 658ms/step
Epoch 5/100
24/24 - 16s - loss: 0.0818 - accuracy: 0.9857 - val_loss: 0.0763 - val_accuracy: 0.9688 - 16s/epoch - 658ms/step
Epoch 6/100
24/24 - 16s - loss: 0.0663 - accuracy: 0.9870 - val_loss: 0.0805 - val_accuracy: 0.9740 - 16s/epoch - 654ms/step
Epoch 7/100
24/24 - 16s - loss: 0.0644 - accuracy: 0.9870 - val_loss: 0.1149 